# Demonstration: Point Defect Formation Energy

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2016-03-30

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm)

- - -

## Software

This notebook was tested with:

- LAMMPS (Version 2016-03-11)

- Python (Version 2.7.11)

- IPython (Version 4.0.3)

- - -

## Introduction

This Notebook calculates the formation energy of different point defects.  To do so, various system sizes are explored and the energy values are extrapolated to infinite atom systems. 

- - -

## 1. Initial Setup

1. __Necessary Python Libraries__: This is a list of the Python libraries used to run this code. 

2. __Necessary Parameters__: These are the parameters that need to be specified to run the code.

3. __Calculation Parameters__: These are user-specified parameters that control how the calculation proceeds, such as number of data points.

### 1.1 Necessary Python Libraries

This is a list of the Python libraries used to run this code. 

- [numpy](http://www.numpy.org/) (Version 1.10.04)

- [atomman](https://github.com/usnistgov/atomman) (Version 0.6)

- [DataModelDict](https://github.com/usnistgov/DataModelDict) (Version 0.8)

In [1]:
#Standard Python Libraries
import os
from copy import deepcopy
import sys

#Additional Python Libraries
import numpy as np
from DataModelDict import DataModelDict
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

### 1.2 Necessary Parameters

These are the parameters that need to be specified to run the code.

- __lammps_exe__: file location for the LAMMPS executable to use.

- __working_dir__: working directory where you want this Notebook to run.

- __potential_file__: file location of the interatomic potential data model file associated with the potential you want to use. A default potential (2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json) is embedded in Section 2.1 of this Notebook, and the associated interatomic potential data model file is embedded in Section 2.2. 

- __potential_dir__: directory location where any potential artifacts (i.e. eam setfl, meam library files) are located.

- __crystal_file__: file location of a crystal prototype data model file or CIF crystal structure file. One example (A1--Cu--fcc.json) is embedded in Section 2.3. 

- __symbols__: list of the potential's atomic symbols to be associated with the lattice sites of the crystal. 

- __a, b, c, alpha, beta, gamma__: specific lattice parameters to use for a unit cell of the crystal.

In [2]:
#Specify LAMMPS run command
lammps_exe = 'C:\\users\\lmh1\\Documents\\lmp_serial.exe'

#Specify working directory
working_dir = 'calc_point_defect_formation'

#This prevents recursive directories upon re-running this cell
if os.path.basename(os.getcwd()) == working_dir:
    working_dir = os.getcwd()
else:
    working_dir = os.path.realpath(working_dir)

#2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json is created in section 2.2
potential_file = os.path.join(working_dir, '2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json')

#MEAM library and potential files for 2012--Jelinek-B--Al-Si-Mg-Cu-Fe
#potential are created in section 2.1
potential_dir = working_dir

#A1--Cu--fcc.json is created in section 2.3
crystal_file = os.path.join(working_dir, 'A1--Cu--fcc.json')

#List the element symbol(s) to use
symbols = ['AlS']

#Give the lattice parameters
a = b = c = 4.0500
alpha = beta = gamma = 90.0

try:
    os.chdir(working_dir)
except:
    os.mkdir(working_dir)
    os.chdir(working_dir)

### 1.3 Calculation Parameters

These are user-specified parameters that control how the calculation proceeds, such as number of data points.

- __point_defect_files__: list of file locations for point defect parameters associated with the specified crystal.  Default models are embedded in Section 2.4 of this notebook associated with the default crystal file. These are placed into the 'A1--Cu--fcc' directory.

- __simulation_box_size__: even integer multiples of the lattice constants to use for the total system box size.

In [3]:
#Specify point defect file(s) 
point_defects_files = [os.path.join('A1--Cu--fcc', 'vacancy.json'),
                       os.path.join('A1--Cu--fcc', '1nn-divacancy.json'),
                       os.path.join('A1--Cu--fcc', '2nn-divacancy.json'),
                       os.path.join('A1--Cu--fcc', 'octahedral-interstitial.json'),
                       os.path.join('A1--Cu--fcc', 'tetrahedral-interstitial.json'),
                       os.path.join('A1--Cu--fcc', 'crowdion-interstitial.json'),
                       os.path.join('A1--Cu--fcc', '100-dumbbell.json'),
                       os.path.join('A1--Cu--fcc', '110-dumbbell.json'),
                       os.path.join('A1--Cu--fcc', '111-dumbbell.json')]

#Specify the system box size 
simulation_box_size = [8, 8, 8]

## 2. Additional Data Files

To make this notebook self-contained, we have embedded a few files.

1. __MEAM Potential.__ Generates library and parameter files for one MEAM potential.

2. __Potential Data Model.__ Collects all input parameters assoiciated with running the MEAM potential in LAMMPS.

3. __Crystal Prototype Data Model.__ Generates an instance of the crystal prototype data model used to generate various structures.

4. __Point Defect Data Models.__ Collects input parameters associated with generating a number of different point defects for the crystal prototype.

### 2.1 MEAM Potential

This generates the library and parameter files associated with the following interatomic potential taken from the [NIST Interatomic Potential Repository](http://www.ctcms.nist.gov/potentials/):

__2012--Jelinek-B--Al-Si-Mg-Cu-Fe__

__Aluminum, Silicon, Magnesium, Copper, and Iron (Al, Si, Mg, Cu, and Fe) Alloys__

*B. Jelinek, S. Groh, M. Horstemeyer, J. Houze, S.G. Kim, G.J. Wagner, A. Moitra, and M.I. Baskes, "Modified embedded atom method potential for Al, Si, Mg, Cu, and Fe alloys," Phys. Rev. B 85, 245102 (2012).*

__Notes__: This file was provided by Bohumir Jelinek (Mississippi State University) and posted on 3 July 2012. He noted, "This is a MEAM potential for Al, Si, Mg, Cu, Fe alloys. It works with LAMMPS, version 19 Jul 2011 or later, when compiled with MEAM support."

__Notice__: Users should consider all available potentials and select one which is appropriate for their use case. Use of this potential within this demonstration should not be construed as an endorsement or a recommendation. 

In [4]:
#Create MEAM library and parameter files for the 2012--Jelinek-B--Al-Si-Mg-Cu-Fe potential
   
#Create MEAM library file "Jelinek_2012_meamf"
f = open('Jelinek_2012_meamf', 'w')
f.write("""# MEAM Al, Si, Mg, Cu, Fe alloy potential, Phys. Rev. B 85, 245102 (2012)
# http://dx.doi.org/10.1103/PhysRevB.85.245102
# to be used with "meam.alsimgcufe" file providing combinational parameters
#  elt        lat     z       ielement     atwt
#  alpha      b0      b1      b2           b3    alat    esub    asub
#  t0         t1              t2           t3            rozero  ibar

'AlS'        'fcc'   12.     13           26.9815
4.64        2.04    3.00    6.0          1.50  4.05    3.353    1.07
1.0         +4.50           -2.30        8.01          1.0    -5
'SiS'        'dia'   4.      14           28.086
4.87        4.4     5.5     5.5          5.5   5.431   4.63    1.
1.0         2.05            4.47         -1.80         2.2    -5
'MgS'       'hcp'   12.      12           24.305
5.52        4.0    3.0     0.2          1.2  3.194 1.51     0.80
1.0         10.04           9.49         -4.3          0.63   -5
'CuS'        'fcc'   12.     29           63.54
5.11        3.634   2.20    6            2.20  3.62    3.54    1.07
1.0         4.91            2.49         2.95          1.10   -5
'FeS'        'bcc'   8       26           55.847
5.0270      3.500   2       1.00         1     2.851   4.28    0.5550
1          -1.6             12.5          -1.40        1.0    -5""")
f.close()

#Create MEAM parameter file "Jelinek_2012_meam.alsimgcufe"
f = open('Jelinek_2012_meam.alsimgcufe', 'w')
f.write("""# MEAM Al, Si, Mg, Cu, Fe alloy potential, Phys. Rev. B 85, 245102 (2012)
# http://dx.doi.org/10.1103/PhysRevB.85.245102
# to be used with "meamf" file providing single element parameters

  Cmin(1,1,1) = 0.8
  repuls(1,1) = 0.1
  Cmin(3,3,3) = 0.8
  Cmin(4,4,4) = 0.8
  Cmin(5,5,5) = 0.68
  repuls(5,5) = 0.3
  Cmax(5,5,5) = 1.9

  nn2(1,1)=1
  nn2(1,2)=1
  nn2(1,3)=1
  nn2(1,4)=1
  nn2(1,5)=1
  nn2(2,2)=1
  nn2(2,3)=1
  nn2(2,4)=1
  nn2(2,5)=1
  nn2(3,3)=1
  nn2(3,4)=1
  nn2(3,5)=1
  nn2(4,4)=1
  nn2(4,5)=1
  nn2(5,5)=1

  lattce(1,2)='b1'
  delta(1,2)=+0.28
  alpha(1,2)=4.56
  re(1,2)=2.62
  Cmin(1,1,2) = 0.50
  Cmin(2,2,1) = 2.00
  Cmin(1,2,1) = 2.00
  Cmin(1,2,2) = 2.00

  lattce(1,3)='b1'
  delta(1,3)=+0.23
  alpha(1,3)=4.52
  re(1,3)=2.87
  Cmin(1,1,3) = 2.00
  Cmin(3,3,1) = 0.00
  Cmin(1,3,1) = 2.00
  Cmin(1,3,3) = 0.00

  lattce(1,4)='b1'
  delta(1,4)=+0.19
  alpha(1,4)=4.65
  re(1,4)=2.53
  Cmin(1,1,4) = 0.00
  Cmin(4,4,1) = 2.00
  Cmin(1,4,1) = 2.00
  Cmin(1,4,4) = 2.00

  lattce(1,5)='b1'
  delta(1,5)=+0.26
  alpha(1,5)=4.64
  re(1,5)=2.45
  Cmin(1,1,5) = 0.90
  Cmin(5,5,1) = 0.10
  Cmin(1,5,1) = 2.00
  Cmin(1,5,5) = 2.00

  lattce(2,3)='b1'
  delta(2,3)=+0.2
  alpha(2,3)=4.73
  re(2,3)=2.75
  Cmin(2,2,3) = 1.00
  Cmin(3,3,2) = 1.00
  Cmin(2,3,2) = 2.00
  Cmin(2,3,3) = 2.00

  lattce(2,4)='b1'
  delta(2,4)=+0.14
  alpha(2,4)=4.74
  re(2,4)=2.46
  Cmin(2,2,4) = 0.00
  Cmin(4,4,2) = 0.00
  Cmin(2,4,2) = 2.00
  Cmin(2,4,4) = 2.00

  lattce(2,5)='b1'
  delta(2,5)=-0.07
  alpha(2,5)=5.17
  re(2,5)=2.39
  Cmin(2,2,5) = 1.00
  Cmin(5,5,2) = 1.00
  Cmin(2,5,2) = 2.00
  Cmin(2,5,5) = 0.00
  attrac(2,5) = 0.1
  repuls(2,5) = 0.1

  lattce(3,4)='b1'
  delta(3,4)=+0.23
  alpha(3,4)=4.70
  re(3,4)=2.63
  Cmin(3,3,4) = 2.00
  Cmin(4,4,3) = 0.00
  Cmin(3,4,3) = 2.00
  Cmin(3,4,4) = 2.00

  lattce(3,5)='b1'
  delta(3,5)=+0.6
  alpha(3,5)=4.96
  re(3,5)=2.61
  Cmin(3,3,5) = 0.65
  Cmin(5,5,3) = 0.00
  Cmin(3,5,3) = 2.00
  Cmin(3,5,5) = 2.00

  lattce(4,5)='b1'
  delta(4,5)=+0.63
  alpha(4,5)=5.21
  re(4,5)=2.42
  Cmin(5,5,4)=0.00

  attrac(5,2) = 0.1
  repuls(5,2) = 0.1

  rc = 5.0
  ialloy=1
  augt1=0
  delr=0.25658351
  emb_lin_neg=1
  bkgd_dyn=1""")
f.close()

### 2.2 Potential Data Model

This generates the interatomic potential data model file associated with the 2012--Jelinek-B--Al-Si-Mg-Cu-Fe potential.

In [5]:
#Creates a interatomic potential data model for using the MEAM potential in LAMMPS
f = open('2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json', 'w')
f.write("""{
    "LAMMPS-potential": {
        "potential": {
            "key": "1515dd80-1984-49de-bc92-d5724059ff56", 
            "id": "2012--Jelinek-B--Al-Si-Mg-Cu-Fe"
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": [
            {
                "symbol": "AlS",
                "element": "Al", 
                "mass": 26.9815                
            }, 
            {
                "symbol": "SiS",
                "element": "Si", 
                "mass": 28.086                
            }, 
            {
                "symbol": "MgS", 
                "element": "Mg",
                "mass": 24.305                
            }, 
            {
                "symbol": "CuS", 
                "element": "Cu", 
                "mass": 63.54
            }, 
            {
                "symbol": "FeS", 
                "element": "Fe", 
                "mass": 55.847
            }
        ], 
        "pair_style": {
            "type": "meam"
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "Jelinek_2012_meamf"
                }, 
                {
                    "symbolsList": "True"
                }, 
                {
                    "file": "Jelinek_2012_meam.alsimgcufe"
                }, 
                {
                    "symbols": "True"
                }
            ]
        }
    }
}""")
f.close() 

### 2.3 Prototype Data Model

This generates the crystal prototype data model file associated with the fcc crystal prototype.

In [6]:
#Creates an instance of a crystal prototype data model for generating crystal structures.
f = open('A1--Cu--fcc.json', 'w')
f.write("""{
    "crystal-prototype": {
        "identifier": {
            "common": "face-centered cubic", 
            "tag": "fcc", 
            "prototype": "Cu", 
            "Pearson-symbol": "cF4", 
            "Strukturbericht": "A1"
        }, 
        "space-group": {
            "number": 225, 
            "Hermann-Maguin": "F m -3 m", 
            "Schoenflies": "O^5_h", 
            "Wykoff": {
                "letter": "a", 
                "multiplicity": 4
            }
        }, 
        "atomic-system": {
            "cell": {
                "cubic": {
                    "a": {
                        "value": 1.0, 
                        "unit": "scaled"
                    }
                }
            }, 
            "atom": [
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.0, 
                            0.0, 
                            0.0
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.0, 
                            0.5, 
                            0.5
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.5, 
                            0.0, 
                            0.5
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.5, 
                            0.5, 
                            0.0
                        ], 
                        "unit": "scaled"
                    }
                }
            ]
        }
    }
}""")
f.close()

### 2.4 Point Defect Data Models

This generates the point defect data model files associated with point defects in the fcc crystal. Each file simply consists of the name of the crystal structure file it is associated with, a name for the defect type, and the atomman.defect.point() function parameters.

In [7]:
#Create A1--Cu--fcc directory for point defects
if not os.path.isdir('A1--Cu--fcc'):
    os.mkdir('A1--Cu--fcc')

with open(os.path.join('A1--Cu--fcc', 'vacancy.json'), 'w') as f:
    f.write("""{
    "point-defect": {
        "crystal-info": {
            "artifact": "A1--Cu--fcc.json"
        }, 
        "identifier": {
            "name": "vacancy"
        }, 
        "atomman-defect-point-parameters": {
            "ptd_type": "v", 
            "pos": [
                0.0, 
                0.0, 
                0.0
            ]
        }
    }
}""")

with open(os.path.join('A1--Cu--fcc', '1nn-divacancy.json'), 'w') as f:
    f.write("""{
    "point-defect": {
        "crystal-info": {
            "artifact": "A1--Cu--fcc.json"
        }, 
        "identifier": {
            "name": "1nn divacancy"
        }, 
        "atomman-defect-point-parameters": [
            {
                "ptd_type": "v", 
                "pos": [
                    0.0, 
                    0.0, 
                    0.0
                ]
            }, 
            {
                "ptd_type": "v", 
                "pos": [
                    0.0, 
                    0.5, 
                    0.5
                ]
            }
        ]
    }
}""")
    
with open(os.path.join('A1--Cu--fcc', '2nn-divacancy.json'), 'w') as f:
    f.write("""{
    "point-defect": {
        "crystal-info": {
            "artifact": "A1--Cu--fcc.json"
        }, 
        "identifier": {
            "name": "2nn divacancy"
        }, 
        "atomman-defect-point-parameters": [
            {
                "ptd_type": "v", 
                "pos": [
                    0.0, 
                    0.0, 
                    0.0
                ]
            }, 
            {
                "ptd_type": "v", 
                "pos": [
                    0.0, 
                    0.0, 
                    1.0
                ]
            }
        ]
    }
}""")
    
with open(os.path.join('A1--Cu--fcc', 'octahedral-interstitial.json'), 'w') as f:
    f.write("""{
    "point-defect": {
        "crystal-info": {
            "artifact": "A1--Cu--fcc.json"
        }, 
        "identifier": {
            "name": "octahedral interstitial"
        }, 
        "atomman-defect-point-parameters": {
            "ptd_type": "i", 
            "atype": 1, 
            "pos": [
                0.5, 
                0.5, 
                0.5
            ]
        }
    }
}""")
    
with open(os.path.join('A1--Cu--fcc', 'tetrahedral-interstitial.json'), 'w') as f:
    f.write("""{
    "point-defect": {
        "crystal-info": {
            "artifact": "A1--Cu--fcc.json"
        }, 
        "identifier": {
            "name": "tetrahedral interstitial"
        }, 
        "atomman-defect-point-parameters": {
            "ptd_type": "i", 
            "atype": 1, 
            "pos": [
                0.25, 
                0.25, 
                0.25
            ]
        }
    }
}""")
    
with open(os.path.join('A1--Cu--fcc', 'crowdion-interstitial.json'), 'w') as f:
    f.write("""{
    "point-defect": {
        "crystal-info": {
            "artifact": "A1--Cu--fcc.json"
        }, 
        "identifier": {
            "name": "crowdion interstitial"
        }, 
        "atomman-defect-point-parameters": {
            "ptd_type": "i", 
            "atype": 1, 
            "pos": [
                0.25, 
                0.25, 
                0.0
            ]
        }
    }
}""")
    
with open(os.path.join('A1--Cu--fcc', '100-dumbbell.json'), 'w') as f:
    f.write("""{
    "point-defect": {
        "crystal-info": {
            "artifact": "A1--Cu--fcc.json"
        }, 
        "identifier": {
            "name": "[100] dumbbell"
        }, 
        "atomman-defect-point-parameters": {
            "ptd_type": "db", 
            "atype": 1, 
            "pos": [
                0.0, 
                0.0, 
                0.0
            ], 
            "db_vect": [
                0.0, 
                0.0, 
                0.3333333333333
            ]
        }
    }
}""")
    
with open(os.path.join('A1--Cu--fcc', '110-dumbbell.json'), 'w') as f:
    f.write("""{
    "point-defect": {
        "crystal-info": {
            "artifact": "A1--Cu--fcc.json"
        }, 
        "identifier": {
            "name": "[110] dumbbell"
        }, 
        "atomman-defect-point-parameters": {
            "ptd_type": "db", 
            "atype": 1, 
            "pos": [
                0.0, 
                0.0, 
                0.0
            ], 
            "db_vect": [
                0.0, 
                0.25, 
                0.25
            ]
        }
    }
}""")
    
with open(os.path.join('A1--Cu--fcc', '111-dumbbell.json'), 'w') as f:
    f.write("""{
    "point-defect": {
        "crystal-info": {
            "artifact": "A1--Cu--fcc.json"
        }, 
        "identifier": {
            "name": "[111] dumbbell"
        }, 
        "atomman-defect-point-parameters": {
            "ptd_type": "db", 
            "atype": 1, 
            "pos": [
                0.0, 
                0.0, 
                0.0
            ], 
            "db_vect": [
                0.25, 
                0.25, 
                0.25
            ]
        }
    }
}""")    

## 3. LAMMPS Script Generation Function(s)

This code generates the underlying LAMMPS script(s) for performing the necessary simulations. For all scripts, system_info and pair_info are required.  system_info is the LAMMPS script lines associated with either generating or reading in atomic system information.  pair_info is the LAMMPS script lines that assigns the atomic interaction model information.  

- min_script() performs an energy minimization on a system. The function allows for the minimization parameters to be controlled. 

In [8]:
def min_script(system_info, pair_info, etol = 0, ftol = 1e-6, maxiter = 100000, maxeval = 100000):
    """Perform energy minimization on atoms in file read_data using potential information from all other parameters."""
    nl = '\n'
    script = nl.join([system_info,
                      '',
                      pair_info,
                      '',
                      'variable peatom equal pe/atoms',
                      '',
                      'thermo_style custom step lx pxx pe v_peatom',
                      'thermo_modify format float %.13e',
                      '',
                      'dump dumpit all custom %i atom.* id type x y z'%(maxeval),
                      'dump_modify dumpit format "%i %i %.13e %.13e %.13e"',
                      'minimize %f %f %i %i'%(etol, ftol, maxiter, maxeval)])
    return script

## 4. Run Calculation

### 4.1 Create System

In [9]:
#Read crystal file and generate unit cell, ucell
with open(crystal_file) as f:
    try:
        ucell = am.models.crystal(f)[0]
    except:
        try:
            ucell = am.models.cif_ucell(f)[0]
        except:
            raise ValueError('Unable to build unit cell from crystal_file.')

#scale ucell by the specified lattice parameters
ucell.box_set(a=a, b=b, c=c, alpha=alpha, beta=beta, gamma=gamma, scale=True)

#Create larger system using simulation_box_size and ucell
assert simulation_box_size[0] % 2 == 0, 'simulation_box_size values must be even integers'
a_size = (-simulation_box_size[0] / 2, simulation_box_size[0] / 2)
assert simulation_box_size[1] % 2 == 0, 'simulation_box_size values must be even integers'
b_size = (-simulation_box_size[1] / 2, simulation_box_size[1] / 2)
assert simulation_box_size[2] % 2 == 0, 'simulation_box_size values must be even integers'
c_size = (-simulation_box_size[2] / 2, simulation_box_size[2] / 2)

system = am.tools.supersize(ucell, a_size, b_size, c_size)

print system

avect =  [32.400,  0.000,  0.000]
bvect =  [ 0.000, 32.400,  0.000]
cvect =  [ 0.000,  0.000, 32.400]
origin = [-16.200, -16.200, -16.200]
natoms = 2048
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 | -16.200 | -16.200 | -16.200
      1 |       1 | -16.200 | -14.175 | -14.175
      2 |       1 | -14.175 | -16.200 | -14.175
      3 |       1 | -14.175 | -14.175 | -16.200
      4 |       1 | -12.150 | -16.200 | -16.200
      5 |       1 | -12.150 | -14.175 | -14.175
      6 |       1 | -10.125 | -16.200 | -14.175
      7 |       1 | -10.125 | -14.175 | -16.200
      8 |       1 |  -8.100 | -16.200 | -16.200
      9 |       1 |  -8.100 | -14.175 | -14.175
     10 |       1 |  -6.075 | -16.200 | -14.175
     11 |       1 |  -6.075 | -14.175 | -16.200
     12 |       1 |  -4.050 | -16.200 | -16.200
     13 |       1 |  -4.050 | -14.175 | -14.175
     14 |       1 |  -2.025 | -16.200 | -14.175
     15 |       1 |  -2.025 | -14.175 | -16.200
     16 |       1 |

### 4.2 Relax Defect Free System

In [10]:
#Read in potential
with open(potential_file) as f:
    potential = lmp.Potential(f, potential_dir)
pair_info = potential.pair_info(symbols)
    
#write system to data file
system_info = lmp.atom_data.dump('perfect.dat', 
                                 system, 
                                 units=potential.units, 
                                 atom_style=potential.atom_style)

#create LAMMPS input script
with open('relax.in', 'w') as f:
    f.write(min_script(system_info, pair_info))

#run lammps
data = lmp.run(lammps_exe, 'relax.in')

#extract cohesive energy
unit_energy = lmp.style.unit(potential.units)['energy']

e_coh = uc.set_in_units(data.finds('peatom')[-1], unit_energy)
print 'Cohesive energy of perfect system:'
print uc.get_in_units(e_coh, 'eV'), 'eV'

#read relaxed system back in
system = lmp.atom_dump.load('atom.'+str(int(data.finds('Step')[-1])))

Cohesive energy of perfect system:
-3.35300001418 eV


### 4.3 Create and Relax Defect Systems

In [12]:
defect_energies = DataModelDict()

#Loop over all defect files
for defect_file in point_defects_files:
    defect_file_name = os.path.basename(defect_file).split('.')[0]
    
    #Open and read the defect file
    with open(defect_file) as f:
        defect = DataModelDict(f)
    
    #Check that the defect is associated with the crystal structure file
    if defect.find('artifact') != os.path.basename(crystal_file):
        print defect.find('artifact'), os.path.basename(crystal_file)
    
    defect_name = defect.find('name')
    
    #create defect system using
    d_system = deepcopy(system)
    for params in defect.iterfinds('atomman-defect-point-parameters'):
        
        #Unscale vector parameters relative to ucell
        if 'pos' in params:
            params['pos'] = ucell.unscale(params['pos'])
        if 'db_vect' in params:
            params['db_vect'] = ucell.unscale(params['db_vect'])
        
        #Send params to point() as keyword arguments
        d_system = am.defect.point(d_system, **params)
        
    #write d_system to data file
    system_info = lmp.atom_data.dump(defect_file_name + '.dat', 
                                     d_system, 
                                     units=potential.units, 
                                     atom_style=potential.atom_style)
    
    #create LAMMPS input script
    with open('relax.in', 'w') as f:
        f.write(min_script(system_info, pair_info))

    #run lammps
    data = lmp.run(lammps_exe, 'relax.in')

    #extract per-atom energy and caclulate formation energy
    d_pot_eng = uc.set_in_units(data.finds('PotEng')[-1], unit_energy)
    defect_energies[defect_name] = d_pot_eng - e_coh * d_system.natoms
    
    #rename relaxed system dump files
    try:
        os.rename('atom.'+str(int(data.finds('Step')[-1])), defect_file_name + '.dump')
    except:
        os.remove(defect_file_name + '.dump')
        os.rename('atom.'+str(int(data.finds('Step')[-1])), defect_file_name + '.dump')

### 4.4 Print Calculated Energy Values

In [13]:
print 'Point Defect Formation Energies'
print 'NOTE: SOME MAY HAVE CHANGED CONFIGURATION!'
print
for name, value in defect_energies.iteritems():
    print '%-30s %.4f eV' % (name+':', uc.get_in_units(value, 'eV'))
    

Point Defect Formation Energies
NOTE: SOME MAY HAVE CHANGED CONFIGURATION!

vacancy:                       0.6693 eV
1nn divacancy:                 1.4084 eV
2nn divacancy:                 1.5431 eV
octahedral interstitial:       3.2759 eV
tetrahedral interstitial:      3.3309 eV
crowdion interstitial:         2.7970 eV
[100] dumbbell:                2.7789 eV
[110] dumbbell:                2.7970 eV
[111] dumbbell:                2.7970 eV


### 4.5 Check Final System Configurations

These are some simple algorithms for checking if a created point defect has relaxed to a different configuration.  Note that these checks do not guarantee a configuration to be stable, and may or may not work depending on the crystal structure and type of defect.

1. __centrosummation__: For simple crystals, the location of a defect is associated with specific lattice positions. As such, these defect positions are symmetry sites.  This does a simple check if the symmetry of the site is broken by summing up all the vector positions of the atoms near the defect relative to the defect's central position. 

2. __defect shift__: Reconfiguration of interstitial and substitutional types of point defects can be investigated by comparing the position of the point defect before and after relaxation.  Defect shift simply is the vector shift of the point defect due to relaxation.

3. __dumbbell vector shift__: Dumbbell defects can rotate from one orientation to another without breaking the centrosummation.  This checks for that by comparing the normalized dumbbell vector before and after relaxation.  

If any of these three vector metrics are not close to zero, then it indicates that a reconfiguration has occured.

In [14]:
-

vacancy
centrosummation =       [ 0.0000, -0.0000,  0.0000]

1nn divacancy
centrosummation =       [-0.0000, -0.0000,  0.0000]

2nn divacancy
centrosummation =       [ 0.0000, -0.0000,  0.0000]

octahedral interstitial
centrosummation =       [-0.0000, -0.0000, -0.0000]
defect shift =          [ 0.0000,  0.0000,  0.0000]

tetrahedral interstitial
centrosummation =       [-0.0000, -0.0000, -0.0000]
defect shift =          [-0.0000, -0.0000, -0.0000]

crowdion interstitial
centrosummation =       [-0.3316, -0.3316, -0.0000]
Defect has reconfigured!

[100] dumbbell
centrosummation =       [-0.0000,  0.0000,  0.0000]
dumbbell vector shift = [-0.0000,  0.0000,  0.0000]

[110] dumbbell
centrosummation =       [ 0.0000, -0.0000, -0.0000]
dumbbell vector shift = [-0.0000, -0.0000,  0.0000]

[111] dumbbell
centrosummation =       [-0.0000, -0.0000, -0.0000]
dumbbell vector shift = [ 0.5774, -0.1298, -0.1298]
Defect has reconfigured!

